In [6]:
import pandas as pd
from ipywidgets import interactive
import ipywidgets as widgets
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter, HourLocator
from tqdm import tqdm

d = "Data//"
flis = ["LT1", "LT2", "LT3", "LT4", "LT5", "LT6", "LT7", "LT8"]
sheet = ["Leq","L90(LN4)"]
llist = ['Load', 'Unit 1', 'Unit 2', 'Unit 3', 'Unit 4']
load = pd.read_excel(d+"Load.xlsx")
load['Time'] = pd.to_datetime(load['Time'].astype(str))
weather = pd.read_excel(d+"Weather.xlsx")
weather['Time'] = pd.to_datetime(weather['Time'].astype(str))
wdir = ["N","NNE","NE","ENE","E","ESE","SE","SSE","S","SSW","SW","WSW","W","WNW","NW","NNW","---"]
lt = [[[] for i in range(2)] for j in range(8)]
dca, dcb = '07:00', '04:00'
lcrit = 1

for i, j in enumerate(flis):
    for k,l in enumerate(sheet):
        lt[i][k] = pd.read_excel(d+flis[i]+".xlsx", sheet_name=l)
        lt[i][k]['Start Time'] = pd.to_datetime(lt[i][k]['Start Time'].astype(str))
        lt[i][k] = pd.merge_asof(lt[i][k],load,left_on="Start Time", right_on="Time")
        lt[i][k] = lt[i][k].drop(labels=['Time'],axis=1)
        lt[i][k] = pd.merge_asof(lt[i][k],weather,left_on="Start Time", right_on="Time")
        lt[i][k] = lt[i][k].drop(labels=['Time'],axis=1)
        lt[i][k] = lt[i][k].set_index('Start Time')
        lt[i][k] = lt[i][k].tail(lt[i][k].shape[0] - 24) # removing first 6 hours from tail
        lt[i][k] = lt[i][k].head(lt[i][k].shape[0] - 24) # removing first 6 hours from head

%matplotlib widget
fig, ax1 = plt.subplots(1, 1, figsize=(8,4))
def plotline(n, lw, d, c, l, w):
    n = int(n-1)
    ax1.clear()
    if lw == 'Leq':
        lwv = 0
        ax1.set_ylabel('L$_{eq}$ dBA')
    else:
        lwv = 1
        ax1.set_ylabel('L$_{90}$ dBA')
    if d == 'Night':
        st, et  = '23:00', '07:00'
    else:
        st, et = '07:00', '23:00'
    ltall = lt[n][lwv].between_time(st, et) #filtering results for day and night
    ltall = ltall[ltall["Rain"]<0.2]
    ltall = ltall[ltall["Speed"]<=5]
    ltall = ltall[ltall["Dir"].isin(w)]
    ltdcall = ltall.between_time(dca, dcb) #filtering for dawn chorus
    lton, ltoff = ltall[ltall[l]>=lcrit], ltall[ltall[l]<lcrit] #filtering results with and without drax power
    ltdcon,ltdcoff = ltdcall[ltdcall[l]>=lcrit],ltdcall[ltdcall[l]<lcrit] #filtering dawn chorus results with and without drax power
    if c:
        ax1.scatter(ltdcoff.index,ltdcoff['Main'],c='black')
        ax1.scatter(ltdcon.index,ltdcon['Main'],c='red')
    else:
        ax1.scatter(ltoff.index,ltoff['Main'],c='black')
        ax1.scatter(lton.index,lton['Main'],c='red')
    myFmt = DateFormatter('%d.%m | %H:%M')
    hours = HourLocator(interval = 6)
    ax1.xaxis.set_major_locator(hours)
    ax1.xaxis.set_major_formatter(myFmt)
    fig.autofmt_xdate()
    ax1.legend([l +' <'+str(lcrit), l +' >='+str(lcrit)])
    fig.show()
box_layout = widgets.Layout(display='inline-flex',flex_flow='column',align_items='flex-start')
ss = widgets.Select(options=llist,description='Load value: ',disabled=False)
aw = widgets.Select(options=list(range(1,9)),description='LT: ',disabled=False)
lw = widgets.ToggleButtons(options=['Leq','L90'],description='Stat. level: ',disabled=False, layout=dict(width='200px'))
bw = widgets.ToggleButtons(options=['Day','Night'],description='Period: ',disabled=False, layout=dict(width='200px'))
cw = widgets.Checkbox(value=False,description='Filter dawn Chorus',disabled=False)
wd = widgets.SelectMultiple(options=wdir,value=[wdir[0]],description='wind dir', disabled=False)
ui = widgets.VBox([widgets.HBox([lw, bw, cw]),widgets.HBox([aw, ss, wd])],layout=box_layout)
out = widgets.interactive_output(plotline, {'n': aw, 'lw': lw, 'd': bw, 'c': cw, 'l':ss,'w':wd})
display(ui,out)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Output()